Creates the final prediction file to submit to kaggle

In [3]:
import pandas as pd, os, numpy as np
import plotly.express as px
pd.options.display.max_columns = 50
import swifter
import pickle as pkl

In [4]:
with open('../data/P.pkl', 'rb') as handle:
    P = pkl.load(handle)

In [5]:
with open('../data/A.pkl', 'rb') as handle:
    A = pkl.load(handle)

In [6]:
with open('../data/C.pkl', 'rb') as handle:
    C = pkl.load(handle)

In [8]:
pdf = pd.DataFrame()
pdf['score'] = P 
pdf['article_id'] = A
pdf['cust_id'] = C

In [9]:
pdf.head()

,score,article_id,cust_id
0,0.917812,751471001,38
1,0.437557,918522001,38
2,0.797402,909370001,38
3,0.194464,915526001,38
4,0.914215,918292001,38


In [10]:
pdf.head()

,score,article_id,cust_id
0,0.917812,751471001,38
1,0.437557,918522001,38
2,0.797402,909370001,38
3,0.194464,915526001,38
4,0.914215,918292001,38


In [11]:
pdf.shape

(8229367, 3)

In [12]:
pdf.sort_values(['cust_id', 'score'], ascending=[True, False], inplace=True)

In [13]:
pdf.head()

,score,article_id,cust_id
2037329,0.950921,673677002,0
2037325,0.947271,706016001,0
2037323,0.937477,751471043,0
2037340,0.932925,920610005,0
2037341,0.931342,863595004,0


In [16]:
gdf = pdf.groupby('cust_id')['article_id'].agg(lambda x: list(x.values[:12]))

In [18]:
pdf.head()

,score,article_id,cust_id
2037329,0.950921,673677002,0
2037325,0.947271,706016001,0
2037323,0.937477,751471043,0
2037340,0.932925,920610005,0
2037341,0.931342,863595004,0


In [22]:
gdf = gdf.reset_index().copy()

In [23]:
def createPredFormat(x):
    preds_sep_f = ''
    for i in range(len(x)):
        val = '0' + str(x[i])
        if i != len(x) - 1:
            val += ' '
        preds_sep_f += val
    return preds_sep_f

In [24]:
gdf['prediction'] = gdf['article_id'].apply(lambda x: createPredFormat(x))

In [26]:
gdf.drop('article_id', axis=1, inplace=True)

In [27]:
gdf

,cust_id,prediction
0,0,0673677002 0706016001 0751471043 0920610005 08...
1,2,0673677002 0805947001 0706016001 0751471043 07...
2,6,0673677002 0706016001 0797988002 0751471043 08...
3,30,0673677002 0805947001 0706016001 0865929007 07...
4,38,0673677002 0805947001 0706016001 0777148006 07...
...,...,...
189505,1371926,0673677002 0898713001 0706016001 0706016062 07...
189506,1371937,0673677002 0706016001 0751471043 0761269001 08...
189507,1371960,0673677002 0706016001 0706016062 0797988002 07...
189508,1371975,0673677002 0706016001 0706016062 0751471043 08...


In [28]:
mfa = [751471001, 909370001, 918522001, 924243001, 918292001, 915526001,
            448509014, 915529003, 751471043, 706016001, 865799006, 863595006,
            850917001, 762846027, 898694001, 673677002, 714790020, 896152002,
            863583001, 762846031, 919365008, 852584001, 923758001, 715624001]

In [29]:
mfa = createPredFormat(mfa)

In [31]:
with open('../data/cid_map.pkl', 'rb') as handle:
    cid_map = pkl.load(handle)

In [32]:
len(cid_map)

1371980

In [55]:
sdf = pd.read_csv('../data/sample_submission.csv')

In [56]:
sdf['cust_id'] = sdf['customer_id'].apply(lambda x: cid_map[x])

In [57]:
sdf.drop('prediction', axis=1, inplace=True)

In [58]:
sdf = sdf.merge(right=gdf, on='cust_id', how='left')

In [59]:
sdf.isna().sum()

customer_id          0
cust_id              0
prediction     1182470
dtype: int64

In [60]:
sdf['prediction'].fillna(mfa, inplace=True)

In [61]:
sdf.isna().sum()

customer_id    0
cust_id        0
prediction     0
dtype: int64

In [62]:
sdf.drop('cust_id', axis=1, inplace=True)

In [63]:
sdf

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0673677002 0706016001 0751471043 0920610005 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0751471001 0909370001 0918522001 0924243001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0673677002 0805947001 0706016001 0751471043 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0909370001 0918522001 0924243001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471001 0909370001 0918522001 0924243001 09...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0673677002 0706016001 0706016062 0751471043 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0751471001 0909370001 0918522001 0924243001 09...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0673677002 0898713001 0706016001 0797988002 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0751471001 0909370001 0918522001 0924243001 09...


In [64]:
sdf.to_csv('sub_nn_binary.csv', index=False)